In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from PIL import Image
import mediapipe as mp
from gfpgan import GFPGANer

In [14]:
def image_detection(img):
    import mediapipe as mp
    mp_face_detection = mp.solutions.face_detection
    with mp_face_detection.FaceDetection(model_selection = 1, min_detection_confidence = 0.5) as face_detector:
        imported_image = img.copy()
        if imported_image.shape[-1] == 4:
            imported_image = cv2.cvtColor(imported_image, cv2.COLOR_RGBA2RGB)
        results = face_detector.process(imported_image)
        frame_height, frame_width, c = imported_image.shape
        if results.detections:
            l = len(results.detections)
    
            if l == 0 or results.detections == None:
                print("no person is found in the image")
                return -1, -1
            elif l > 1:
                print("multiple persons are found")
                return -1, -1
            for face in results.detections:
                face_react = np.multiply(
                        [
                            face.location_data.relative_bounding_box.xmin,
                            face.location_data.relative_bounding_box.ymin,
                            face.location_data.relative_bounding_box.width,
                            face.location_data.relative_bounding_box.height,
                        ],[frame_width, frame_height, frame_width, frame_height]).astype(int)
                
                cv2.rectangle(imported_image, face_react, color=(255, 255, 255), thickness=2)
                key_points = np.array([(p.x, p.y) for p in face.location_data.relative_keypoints])
                key_points_coords = np.multiply(key_points,[frame_width,frame_height]).astype(int)
                i = 1
                for p in key_points_coords:
                    # if i == 2 or i == 1:
                    #     continue
                    cv2.circle(imported_image, p, 4, (255, 255, 255), 2)
                    cv2.circle(imported_image, p, 2, (0, 0, 0), -1)
                    i += 1
                
                return face_react, key_points_coords

            return -1, -1

        return -1, -1


# one and two are the coordinates for the eyes in the relative_keypoints and this model is identifying the persons accurately

In [15]:

def align_face(image, keypoints):
    left_eye = keypoints['left_eye']
    right_eye = keypoints['right_eye']

    dy = right_eye[1] - left_eye[1]
    dx = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dy, dx))

    center_x, center_y = (int((left_eye[0] + right_eye[0]) // 2), int((left_eye[1] + right_eye[1]) // 2))

    rotation_matrix = cv2.getRotationMatrix2D((center_x, center_y), angle=angle, scale=1.0)
    aligned_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))

    return aligned_image


In [16]:
#lets do a little experimentation to find the exact eye pointings in the image in relative key points
def align_detect_image(img, keypoints):
    aligned_image = align_face(img, keypoints)
    # plt.imshow(aligned_image)
    r2, c2 = image_detection(aligned_image)
    # print(r2, c2)
    return aligned_image, r2



In [7]:
from keras_facenet import FaceNet
embedder = FaceNet()

In [17]:
def enhance_faces(input_img):
    # Use the correct variable for input image
    result = gfpganer.enhance(input_img, has_aligned=False, only_center_face=False)
    
    if isinstance(result, tuple):
        restored_image = result[1][0]  # Access the first restored image
        if isinstance(restored_image, np.ndarray):
            # Ensure the restored image has correct data type
            if restored_image.dtype != np.uint8:
                restored_image = np.clip(restored_image, 0, 255).astype(np.uint8)
        return restored_image
    else:
        print("Unexpected result format from GFPGAN.")
        return None

In [18]:
base_path = os.getcwd()
# inter_path = os.path.join(base_path, "models");
model_path = r"D:\venv's\frs\models\GFPGANv1.4.pth"
gfpganer = GFPGANer(
    model_path=model_path,
    upscale=2,  # Upscaling factor
    arch="clean",  # For general restoration
    channel_multiplier=2  # Default channel multiplier
)

D:\venv's\frs\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\venv's\frs\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
D:\venv's\frs\Lib\site-packages\facexlib\detection\__init__.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limi

In [19]:
# now we can write a function for similarity between the embeddings in two images , based on that we can set certain treshold for that
#lets find the embedding of two similar images and calculate the cosine between them

def cosine_similarity(embedding1, embedding2):

    embedding1 = embedding1.flatten()
    embedding2 = embedding2.flatten()


    dot_product = np.dot(embedding1, embedding2)
    norm_embedding1 = np.linalg.norm(embedding1)
    norm_embedding2 = np.linalg.norm(embedding2)
    similarity = dot_product / (norm_embedding1 * norm_embedding2)
    return similarity

In [22]:
def get_embedding(img):
    new_img = img.copy()
    results, coords = image_detection(new_img)
    if type(results) == int:
        return -1

    keypoints = {}
    keypoints['left_eye'] = coords[0]
    keypoints['right_eye'] = coords[1]
    
    aligned_image, coords = align_detect_image(new_img, keypoints)
    x, y, h, w = coords
    cropped_img = aligned_image[y:y+h, x:x+w]
    enhanced_img = enhance_faces(cropped_img)
    return embedder.embeddings([enhanced_img])
        

In [23]:
img = plt.imread("enhanced_images/Ashish.jpg")
print(get_embedding(img))

1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
[[ 4.28235605e-02  4.34188806e-02 -2.40854104e-03 -3.08456272e-02
   9.16924793e-03 -2.06387993e-02 -1.42453471e-02  4.50746343e-03
  -2.74597798e-02  6.40464202e-02  2.78751971e-03 -1.82694774e-02
  -2.43657548e-02 -1.19122416e-02  7.92349428e-02 -3.26671056e-03
   2.17427984e-02  4.67558615e-02 -2.76876129e-02  5.57158664e-02
   2.24827835e-03  1.86423846e-02  6.64270446e-02 -5.40537909e-02
   1.02979289e-02  1.38935568e-02 -3.13225016e-02 -7.29916096e-02
  -2.07244568e-02 -1.16373028e-03 -5.86510338e-02  5.57384118e-02
  -2.72515863e-02  6.54084384e-02  7.73771256e-02 -8.80002230e-03
   6.07837811e-02 -9.97836217e-02 -2.35959534e-02 -1.40898358e-02
   1.88476834e-02  1.32703912e-02 -6.57716859e-03 -1.16260108e-02
  -7.81895444e-02  2.35042092e-03 -2.10288949e-02  2.17109136e-02
  -8.02127272e-02  8.65945313e-03  4.67942283e-03 -5.60582764e-02
  -1.23415445e-03  3.26069333e-02 -5.16512729e-02 -1.12356007e-01
  -4.59757708e-02  7.93318599e-02 -3.9